In [2]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [7]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [8]:
key = 'sk' ## set openai api key

In [9]:
import os
os.environ['OPENAI_API_KEY'] = key

In [10]:
pdfreader = PdfReader('/content/72996532540_17_06_22.pdf')

In [11]:
from typing_extensions import Concatenate

raw_text = ''
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [12]:
raw_text

" \n  \n  \nTATA Consultancy Services Limited  \n9th Floor Nirmal Building Nariman Point Mumbai 400 021  \nTel. 91 22 6778 9595 Fax 91 22 6778 9660 e -mail corporate.office@tcs.com   website www.tcs.com  \nRegistered Office 9th Floor Nirmal Building Nariman Point Mumbai 400 021.  \nCorporate identification No. (CIN): L22210MH1995PLC084781  \n \nTCS/SE/ 44/2022-23 \n \nJune 2, 202 2 \n \nNational Stock Exchange of India Limited    BSE Limited  \nExchange Plaza, C -1, Block G, Bandra Kurla   P. J. Towers,       \nComplex, Bandra (East)      Dalal Street,  \nMumbai - 400051                                                              Mumbai - 400001    \nSymbol - TCS                                                              Scrip Code No. 532540  \n      \nDear Sirs,  \n \nSub: Intimation under Regulation 34 of SEBI (Listing Obligations and Disclosure  \nRequirements) Regulations, 2015  \n \nThis is further to our letter dated May 18, 202 2 wherein the Company has filed with the stock 

In [17]:
### Use instead of pdf
## NOTE: Large texts won't work for free trial

raw_text = '''On Halloween, Gotham City mayor Don Mitchell Jr. is murdered by the Riddler, a masked serial killer. Reclusive billionaire Bruce Wayne, who has operated for two years as the vigilante Batman, investigates the murder alongside the Gotham City Police Department (GCPD). Lieutenant James Gordon discovers a message that the Riddler left for Batman. The following night, the Riddler kills Commissioner Pete Savage and leaves another message for Batman.

Batman and Gordon discover that the Riddler left a thumb drive in Mitchell's car containing images of Mitchell with a woman, Annika Kosolov, at the Iceberg Lounge—a nightclub operated by Oswald "The Penguin" Cobblepot, crime boss Carmine Falcone's lieutenant. While the Penguin pleads ignorance, Batman notices that Selina Kyle, Annika's roommate, works at the club as a waitress. When Annika disappears, Batman sends Selina to the Iceberg Lounge for answers and discovers that Savage was on Falcone's payroll, as is district attorney Gil Colson.

The Riddler abducts Colson, straps a timed collar bomb to his neck, and sends him to interrupt Mitchell's funeral. When Batman arrives, the Riddler calls him via Colson's phone and threatens to detonate the bomb if Colson cannot answer three riddles. Colson refuses to answer the third—the name of the informant who gave the GCPD information that led to a historic drug bust ending gangster Salvatore Maroni's operation—and is killed. Batman and Gordon deduce that the informant may be the Penguin and track him to a drug deal. They discover that Maroni's operation transferred to Falcone, with many corrupt GCPD officers involved. Selina inadvertently exposes them when she arrives to steal money and discovers Annika's corpse in the trunk of a car. After a car chase, Batman captures the Penguin, but he and Gordon learn the Penguin is not the informant.

Batman and Gordon follow the Riddler's trail to the ruins of an orphanage funded by Bruce's murdered parents, Thomas and Martha Wayne, where they learn that the Riddler holds a grudge against the Wayne family. Bruce's butler and caretaker, Alfred Pennyworth, is hospitalized after opening a letter bomb addressed to Bruce. The Riddler leaks evidence that Thomas, who was running for mayor before he was murdered, hired Falcone to kill a journalist who was writing a story about Martha's family history of mental illness. Bruce confronts Alfred, who maintains that Thomas only asked Falcone to threaten the journalist into silence; Thomas planned to turn himself and Falcone over to the police once he found out Falcone murdered the journalist instead. Alfred believes that Falcone had Thomas and Martha killed to silence them.

Selina reveals to Batman that Falcone is her father. After learning that he killed Annika for discovering he was the informant, she tries to kill Falcone, but Batman and Gordon arrive in time to stop her. Gordon arrests Falcone, but the Riddler kills him with a sniper rifle moments later. The Riddler is unmasked as forensic accountant Edward Nashton and incarcerated in Arkham State Hospital, where he tells Batman he took inspiration from him when targeting the corrupt. Batman learns that Nashton has planted car bombs around Gotham and cultivated an online following that plans to assassinate mayor-elect Bella Reál.

The bombs destroy the seawall around Gotham and flood the city. Nashton's followers attempt to kill Reál, but Batman and Selina manage to stop them. In the aftermath, Nashton befriends another inmate,[a] while Selina deems Gotham beyond saving and leaves. Batman aids recovery efforts and vows to inspire hope in Gotham.'''

In [18]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

In [19]:
len(texts)

6

In [20]:
embeddings = OpenAIEmbeddings()

In [21]:
document_search = FAISS.from_texts(texts, embeddings)

In [23]:
document_search

In [24]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [25]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [28]:
query = "How many villains are in the movie?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Two villains, the Riddler and the Penguin.'